In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, float64, complex128, int64, types, objmode

from matplotlib.patches import Circle


from collections import namedtuple as nt
mode_params=nt("mode_params", "a0 m p Mp w0")

from scipy.special import hermite, genlaguerre

%matplotlib inline

In [2]:
@jit(types.Tuple((complex128[:,:,:], 
                  float64[:,:,:], 
                  int64, 
                  int64))
     (int64, 
      int64, 
      float64, 
      float64, 
      float64, 
      float64, 
      float64[:,:,:], 
      float64[:,:,:], 
      float64[:,:,:]), cache=True, nopython=True, parallel=True)
def LG_mode_accelerated(mode_params_m, 
                        mode_params_p, 
                        mode_params_Mp, 
                        mode_params_w0, 
                        mode_params_a0, tau, r, phi, z):
    a0 = mode_params_a0
    m = mode_params_m  # mode number x
    p = mode_params_p  # mode number y
    Mp = mode_params_Mp
    w0 = mode_params_w0
    
    #Lpm=genlaguerre(p,np.abs(m))

    ZR = Mp*w0**2/2
    ZR_sq = np.square(ZR)
    r_sq = np.square(r)
    z_sq = np.square(z)
    
    w = w0*np.sqrt(1+z_sq/ZR_sq)
    w_sq = np.square(w)
    chi = np.arctan(z/ZR)
    
    
    #toret = 1/np.sqrt(w)*(r*np.sqrt(2)/w)**np.abs(m)
    toret = (r*np.sqrt(2)/w)**np.abs(m)

    exp_factor = np.exp(-1j*r_sq*z/2/(ZR_sq+z_sq) + 1j*m*phi -z_sq/2/tau**2 + \
                        1j*Mp*z -1j*(2*p+np.abs(m)+1)*chi-r_sq/w_sq)
    toret = toret * exp_factor
    
    return a0*toret, 2*r_sq/w_sq, p, m

In [3]:
class LG_mode_generation():
    
    """
        This class features the suport of LG modes generation and reflected field analysis.
        After fitting, the following results are stored in corresponding member variables:
            self.zmirror - describes the motion of target
            self.refl_field - interpolated field generated after LG_mode interaction with target
            self.fft_result - FFT transform of the reflected field
            self.fft_freq - set of corresponding frequencies in FFT
    """
    
    def __init__(self, params_dict):
        
        self.fitted = False
        self.mode_params = []
        self.mode_params.append(mode_params(*params_dict["mode_0"])) ###mode_params
        self.mode_params.append(mode_params(*params_dict["mode_1"])) ###mode_params
        self.coord_type = params_dict["coords"]["type"]
        
        self.tau = params_dict["tau"]
        self.delay = params_dict["delay"]
        
        if self.coord_type == "cartesian":
            xmin = params_dict["coords"]["xmin"]#-10
            xmax = params_dict["coords"]["xmax"]#10
            ymin = params_dict["coords"]["ymin"]#-10
            ymax = params_dict["coords"]["ymax"]#10
            zmin = params_dict["coords"]["zmin"]#-15
            zmax = params_dict["coords"]["zmax"]#15
            
            nofxs = params_dict["discretization"]["x"]#60
            nofys = params_dict["discretization"]["y"]#60
            nofzs = params_dict["discretization"]["z"]#2400
            
            self.zmin, self.zmax = zmin, zmax
            
            xx = np.linspace(xmin*2*np.pi, xmax*2*np.pi, nofxs)
            yy = np.linspace(ymin*2*np.pi, ymax*2*np.pi, nofys)
            zz = np.linspace(zmin*2*np.pi, zmax*2*np.pi, nofzs)
            self.d_sampling_rate = zz[1] - zz[0]
            self.nofzs = nofzs
            self.x,self.y,self.z = np.meshgrid(xx,yy,zz, indexing='xy')
        elif (self.coord_type == "polar"):
            zmin = params_dict["coords"]["zmin"]#-15
            zmax = params_dict["coords"]["zmax"]#15
            nofzs = params_dict["discretization"]["z"]#2400
            zz = np.linspace(zmin*2*np.pi, zmax*2*np.pi, nofzs)
            self.d_sampling_rate = zz[1] - zz[0]
            rhos = np.linspace(0, 7*2*np.pi, 20)
            phis = np.linspace(0, 2*np.pi, 150)
            
            self.zmin, self.zmax = zmin, zmax
            
            self.r,self.phi,self.z = np.meshgrid(rhos,phis,zz, indexing='xy')
        else:
            raise ValueError("Unsupported coordinate type!\
            Supported values are [\'cartesian\', \'polar\']")
    @staticmethod
    def calculate_spectrum(field, d_rate):
        fft_res = np.fft.fft(field, axis=2)
        fft_freq = np.fft.fftfreq(len(field[1,1,:]), d=d_rate)
        return fft_res, fft_freq
        
    @staticmethod
    def LG_mode(mode_params, tau, r, phi, z):
        mode, arg, p,m = LG_mode_accelerated(mode_params.m, 
                                         mode_params.p, 
                                         mode_params.Mp, 
                                         mode_params.w0, 
                                         mode_params.a0, tau, r, phi, z)
        Lpm = genlaguerre(p, np.abs(m))
        mode = mode * Lpm(arg)
        return mode
    
    @staticmethod
    def get_interp_field(z, zd, refl_field):
        refl_field_interp = np.zeros_like(refl_field)
        for i in range(z.shape[0]):
            for j in range(z.shape[1]):
                refl_field_interp[i,j,:] = np.interp(z[i,j,:], zd[i,j,:], refl_field[i,j,:])
        return refl_field_interp
    
    def coords_polar(self):
        if self.coord_type =="polar":
            return self.r, self.phi
        else:
            r = np.sqrt(self.x**2+self.y**2)

            xiy = self.x + 1j*self.y
            phi = np.angle(xiy)
            return r, phi
        
    def calculate_field(self):
        r, phi = self.coords_polar()
        start = datetime.now()
        print("Calculating LG modes...", end="")
        LG_mode_1 = self.LG_mode(self.mode_params[0], self.tau, r, phi, self.z-self.delay/2)
        field_x = np.abs(LG_mode_1) * np.cos(np.angle(LG_mode_1))

        LG_mode_2 = self.LG_mode(self.mode_params[1], self.tau, r, phi, self.z+self.delay/2)
        field_x = field_x + np.abs(LG_mode_2) * np.cos(np.angle(LG_mode_2))
        
        print("OK!")
        
        envelope1 = np.abs(LG_mode_1)
        envelope2 = np.abs(LG_mode_2)
        
        print("Calculating reflected fields...", end="")
        full_envelope = np.sqrt(envelope1**2 + \
                            envelope2**2 + 2*envelope1*envelope2*\
                            np.cos(np.angle(LG_mode_2)-np.angle(LG_mode_1)))

        zmirror = 1./(1+full_envelope ** 2)*(envelope1*np.cos(np.angle(LG_mode_1))+\
                                                 envelope2*np.cos(np.angle(LG_mode_2)))**2

        refl_field = envelope1 * np.cos(np.angle(self.LG_mode(self.mode_params[0], 
                                                              self.tau, r, phi, self.z-self.delay/2+zmirror)))
        refl_field = refl_field + envelope2 * np.cos(np.angle(self.LG_mode(self.mode_params[1],
                                                              self.tau, r, phi, self.z+self.delay/2+zmirror)))

        zd = self.z-zmirror
        
        self.incident_field = field_x
        self.zmirror = zmirror
        self.full_envelope = full_envelope
        self.refl_field = self.get_interp_field(self.z, zd, refl_field)
        print("OK!")
        print("Calculating field spectrum...", end="")
        self.fft_result, self.fft_freq = self.calculate_spectrum(self.refl_field, 
                                                                 self.d_sampling_rate)
        self.fitted = True
        print(f"OK!\nFinished training in {(datetime.now()-start).total_seconds()} seconds.")
        return True
    def count_spectrogram(self, r, n_harmonics=3):
        dz = self.d_sampling_rate/(2*np.pi)      # in wavelength

        window_width = 2.0   # in wavelengths
        dtau_window = 0.1    # in wavelengths
        nofzs_window = (self.zmax - self.zmin)/dtau_window

        omegas = np.fft.fftfreq(self.refl_field.shape[2], d=dz)

        idx = np.argwhere(np.abs(np.abs(omegas)-n_harmonics)<1e-2)[0][0] #3rd harmonics idx

        print("Counting spectrogram...")
        spectrogram = np.zeros((self.refl_field.shape[1],
                                self.refl_field.shape[0], 
                                int(nofzs_window), 
                                1), 
                               dtype=np.complex128)
        for r in range(self.refl_field.shape[1]):
            for phi in range(self.refl_field.shape[0]):
                field = self.refl_field[phi,r,:]
                #sp=np.fft.fft(field)
                specgram_phi = None
                flag = False
                for n in np.arange(nofzs_window):
                    window_pos = self.zmin+n * dtau_window
                    window_func = np.exp(-((self.z[phi, r,:]/2/np.pi - window_pos)/window_width)**100)
                    y = field * window_func
                    sp = np.fft.fft(y)
                    if flag == False:
                        specgram = sp[idx]
                        flag = True
                    else:
                        specgram = np.vstack([specgram, sp[idx]])
                spectrogram[r, phi, :, :] = specgram
                #specgram_global[r, phi] = specgram 
            #self.specgram = specgram_global
        return spectrogram
    def plot_incident_field(self, ):
        pass
    def calculate_radial_spectrum(self, rho):
        r, phi = self.coords_polar()
        try:
            field_slice = self.incident_field[:,rho, :]
        except:
            print(f"Invalid rho value ({rho})!")
            return
        radial_spectrum = np.fft.fft(field_slice, axis=1)
        frequencies = np.fft.fftfreq(self.nofzs, self.d_sampling_rate)
        return radial_spectrum, frequencies

In [4]:
params_dict = {
    "mode_0" : [3., 1, 0, 1., 3.**2*np.pi],
    "mode_1" : [3., 2, 0, 1., 3.**2*np.pi],
    "coords" :
        {
            "type": "cartesian",
            "xmin": -10,
            "xmax": 10,
            "ymin": -10,
            "ymax": 10,
            "zmin": -15,
            "zmax": 15
        },
    "tau": 4*2*np.pi,
    "delay": -2*4*2*np.pi,
    "discretization": {
                        "x": 60,
                        "y": 60,
                        "z": 2400
    }
               }

In [5]:
params_dict = {
    "mode_0" : [5., 1, 0, 1., 3.**2*np.pi],
    "mode_1" : [5., 2, 0, 1., 3.**2*np.pi/np.sqrt(2.)],
    "coords" :
        {
            "type": "polar",
            "zmin": -15,
            "zmax": 15
        },
    "tau": 8*2*np.pi,
    "delay": -8*2*np.pi,
    "discretization": {
            
                        "z": 2400
    }
               }

In [6]:
lg_mode = LG_mode_generation(params_dict)

In [7]:
lg_mode.calculate_field()

Calculating LG modes...OK!
Calculating reflected fields...OK!
Calculating field spectrum...OK!
Finished training in 3.794796 seconds.


True

In [8]:
lg_mode.refl_field.shape

(150, 20, 2400)

In [9]:
lg_mode.z[0,0,:]/2/np.pi

array([-15.        , -14.98749479, -14.97498958, ...,  14.97498958,
        14.98749479,  15.        ])

In [ ]:
spectrogram = lg_mode.count_spectrogram(r=10, n_harmonics=3)

In [ ]:
np.save("spectrogram_3rd_harmonics.npy", spectrogram[:,:,:,0])

In [10]:
spectrogram = np.load("spectrogram_3rd_harmonics.npy")

In [11]:
spectrogram.shape

(20, 150, 300)

In [12]:
spectrogram.reshape(-1,300)

array([[ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       ...,
       [-2.02410328-0.99929789j, -0.68656522-1.81160119j,
        -0.63642019-2.31051728j, ...,  1.0001851 +1.12705604j,
         1.02926338-1.00875041j, -0.80944943-0.62190123j],
       [-1.87622784-1.16999993j, -0.58154288-1.93559325j,
        -0.44602786-2.39574989j, ...,  0.80661543+1.18381171j,
         0.8669309 -0.98133202j, -0.92692749-0.62446811j],
       [-1.71504527-1.32443016j, -0.46330708-2.04291245j,
        -0.24169131-2.46364652

In [16]:
r = np.linspace(0,19,20)
phi = np.linspace(0,2*np.pi,150)
rhos, phis = np.meshgrid(r,phi)

In [15]:
from ipywidgets import interact
@interact(n=(0,299))
def Plotter(n=0):
    plt.figure(figsize=(4,4))
    plt.gca(projection="polar")
    plt.pcolormesh(phi,r,np.abs(spectrogram[:,:,n]))

interactive(children=(IntSlider(value=0, description='n', max=299), Output()), _dom_classes=('widget-interact'…

In [53]:
refl_field.dtype

dtype('float64')

In [51]:
refl_field = np.transpose(lg_mode.refl_field,[1,0,2])

In [60]:
from mpl_toolkits.mplot3d import Axes3D
@interact(n=(0,2400,5))
def Plotter(n=0):
    fig = plt.figure(figsize=(16,8))
    ax = fig.add_subplot(121, projection='3d')
    ax.set_title(f"3d visualization of spectrogram, slice no. is {n}")
    rhos, phis = np.meshgrid(r,phi)
    X, Y = rhos * np.cos(phis), rhos * np.sin(phis)
    ax.plot_surface(X, Y, refl_field[:,:,n].T, cmap=plt.cm.YlGnBu_r)
    ax.set_xlim(-20,20)
    ax.set_ylim(-20,20)
    ax.set_zlim(-1,1)
    ax = fig.add_subplot(122, projection='polar')
    ax.set_title(f"2d visualization of spectrogram, slice no. is {n}")
    #ax.gca(projection="polar")
    ax.pcolormesh(phi,r,refl_field[:,:,n])

interactive(children=(IntSlider(value=0, description='n', max=2400, step=5), Output()), _dom_classes=('widget-…

In [ ]:
lg_mode.zmirror.shape

# Mirror motion

TO-DO:
 - 3d scape

In [ ]:
@interact(n=(0,2399,5))
def Plotter(n=0):
    plt.figure(figsize=(4,4))
    plt.gca(projection="polar")
    plt.pcolormesh(phis,rhos,lg_mode.zmirror[:,:,n])

In [ ]:
dz = lg_mode.d_sampling_rate/(2*np.pi)      # in wavelength

window_width = 2.0   # in wavelengths
dtau_window = 0.1    # in wavelengths
nofzs_window = (zmax - zmin)/dtau_window


omegas = np.fft.fftfreq(lg_mode.refl_field.shape[2], d=dz)

idx = np.argwhere(np.abs(np.abs(omegas)-3)<1e-2)[0][0] #3rd harmonics idx

for r in range(lg_mode.refl_field.shape[0]):
    print(r)
    for phi in range(lg_mode.refl_field.shape[1]):
        field = lg_mode.refl_field[r,phi,:]
        sp=np.fft.fft(field)
        specgram_global = np.zeros((lg_mode.refl_field.shape[0],
                                    lg_mode.refl_field.shape[0] , 300,1), dtype=np.complex128)
        flag = False
        for n in np.arange(nofzs_window):
            window_pos = zmin+n * dtau_window
            window_func = np.exp(-(z/2/np.pi - window_pos)**100/window_width**100)
            y = field * window_func
            sp = np.fft.fft(y)

            # index 3rd harmonic
            # for example sp[10] corresponds to 3rd harmonic
            # r,phi, angle(sp[10])

            if flag == False:
                specgram = sp[idx]
                flag = True
            else:
                specgram = np.vstack([specgram, sp[idx]])
        #print(specgram.shape)
        specgram_global[r, phi] = specgram 

In [ ]:
window_pos = zmin + 0* dtau_window
window_func = np.exp(-((z/2/np.pi - window_pos)/window_width)**100)

In [ ]:
new_field = lg_mode.refl_field.reshape(-1)

In [ ]:
np.savetxt("field.txt", lg_mode.refl_field.reshape(-1))

In [ ]:
import ctypes

In [ ]:
libc = ctypes.CDLL("./libfilter.so")

#libc.apply_filter.restype = ctypes.c_double # define mean function return type

In [ ]:
libc.mean

In [ ]:
omegas = np.fft.fftfreq(lg_mode.refl_field.shape[2], d = dz)
@jit((complex128[:,:,:,:](float64[:,:,:],float64, float64, float64, float64[:])), cache=True, parallel=True)
def calculate_specgram(refl_field, dz, zmin, zmax, omegas):
    #dz = lg_mode.d_sampling_rate/(2*np.pi)      # in wavelength

    window_width = 2.0   # in wavelengths
    dtau_window = 0.1    # in wavelengths
    nofzs_window = (zmax - zmin)/dtau_window

    flag = False

    
    idx = np.argwhere(np.abs(np.abs(omegas)-3)<1e-2)[0][0] #3rd harmonics idx
    for r in range(refl_field.shape[0]):
        print(r)
        for phi in range(refl_field.shape[1]):
            field = refl_field[r,phi,:]
            sp=np.fft.fft(field)
            for n in np.arange(nofzs_window):
                window_pos = zmin+n * dtau_window
                window_func = np.exp(-(z/2/np.pi - window_pos)**100/window_width**100)
                y = field * window_func
                with objmode(out='complex128[:,:]'):
                    sp = np.fft.fft(y)

                # index 3rd harmonic
                # for example sp[10] corresponds to 3rd harmonic
                # r,phi, angle(sp[10])

                if flag == False:
                    specgram = sp[idx]
                    flag = True
                else:
                    specgram = np.vstack([specgram, sp[idx]])

In [ ]:
calculate_specgram(lg_mode.refl_field,dz,zmin, zmax, omegas)

In [ ]:
nrows = 2306
ncols = 95
Z = np.abs(spectrogram_result)
print(Z.shape)
x = np.arange(ncols + 1)
y = np.arange(nrows + 1)


plt.pcolormesh(x, y, Z,)

In [ ]:
spectrogram_result.shape == Z.shape

In [ ]:
y.shape

In [ ]:
Z.shape

In [ ]:
import example

In [ ]:
%%timeit
b = example.multiply(np.linspace(-15,15,2400),1)

In [ ]:
z = np.linspace(-15,15,2400)
window_pos=-15.
window_width = 2.0


In [ ]:
%%timeit
window_func = np.exp(-(z/2/np.pi - window_pos)**100/window_width**100)